# Analyze Satisfiability Results

This notebook is used to compute mean accuracy in a number of experiments. Each answer file corresponds to a different experiment.

In [1]:
import json

answers = json.load(open('results/answers_gpt35_tempR.json', 'r'))
#answers = json.load(open('results/answers_gpt35_tempG.json', 'r'))
#answers = json.load(open('results/answers_gpt35_cot_1shot.json', 'r'))
#answers = json.load(open('results/answers_gpt35_cot_2shot.json', 'r'))
#answers = json.load(open('results/answers_gpt35_cot_3shot.json', 'r'))
#answers = json.load(open('results/answers_gpt35_cot_4shot.json', 'r'))
#answers = json.load(open('results/answers_gpt35_cot_8shot.json', 'r'))
#answers = json.load(open('results/answers_gpt4_tempR.json', 'r'))
#answers = json.load(open('results/answers_gpt4_tempG.json', 'r'))
print('Read %i responses.' % len(answers))

Read 6080 responses.


## Recode Raw Responses

The raw LLM responses are non-deterministic and may contain commentary or changes in output formatting. Review the raw responses and extract the answers using one of three rules:

1. If the response is True or False, accept it
2. If the response contains one or more "Answer:" trigger word followed by True or False, accept the last instance
3. If the response includes the sentence "The statement is " following by True or False, accept the last instance. This response may also include the requirement with or without quotes in the sentence, which can be ignored.

Finally, count the number of non-uniform responses. Non-uniform responses with no detectable answer are nullified and thus counted as a false positive.

In [2]:
from collections import Counter
import re, copy

p1 = re.compile('Answer: (True|False)', re.IGNORECASE)
p2 = re.compile('the requirement ("?.+?"?\s)?(is|would be) (true|false)', re.IGNORECASE)
p3 = re.compile('True|False', re.IGNORECASE)

count = 0
match_counts = Counter()
mismatch = []
for answer in answers:
    if not answer[5] in ['True', 'False']:
        count += 1
        
        # try to find the answer using the trigger words
        matches = []
        for m in p1.finditer(answer[5]):
            matches.append(m.group(1))
           
        # try to find the answer embedded in a sentence
        if len(matches) == 0:
            for m in p2.finditer(answer[5]):
                if m.group(3).lower() == 'true':
                    matches.append('True')
                else:
                    matches.append('False')
        
        if len(matches) == 0:
            for m in p3.finditer(answer[5]):
                if m.group(0).lower() == 'true':
                    matches.append('True')
                else:
                    matches.append('False')

        if len(matches) == 0:
            mismatch.append(answer[5])
            answer[5] = 'None'
        else:
            answer[5] = matches[-1]
            
        match_counts[len(matches)] += 1
            
print('Found %i non-uniform responses:' % count)
for i in range(0, len(match_counts)):
    print('\t%i answer(s): %i' % (i, match_counts[i]))

Found 42 non-uniform responses:
	0 answer(s): 0
	1 answer(s): 18
	2 answer(s): 0
	3 answer(s): 0
	4 answer(s): 4


In [3]:
print('\n\n'.join(mismatch))

## Research Questions

* RQ1: To what extent does property affect accuracy?
* RQ2: To what extent does only scenario polarity affect accuracy?
* RQ3: To what extent does only question polarity affect accuracy?
* RQ4: To what extent does majority vote affect accuracy?

In [4]:
from collections import Counter
import random

# answer format: spec id, run id, spec property, spec polarity, expected, predicted
# answer format example: S-X-0001, 2, C, T, True, False

# list scenario ids for reporting
scenario_ids = set([a[0] for a in answers])
properties = set([a[2] for a in answers])

# setup frequency counters
freq_overall = {'T': 0, 'F': 0}
freq_property = {p:{'T': 0, 'F': 0} for p in properties}
freq_specification_polarity = {'T': {'T': 0, 'F': 0}, 'F': {'T': 0, 'F': 0}}
freq_requirement_polarity = {'T': {'T': 0, 'F': 0}, 'F': {'T': 0, 'F': 0}}
freq_majority_votes = {}
freq_majority = {'T': 0, 'F': 0}

for answer in answers:
    spec_id = answer[0]
    prop_code = answer[2]
    spec_polarity = answer[3]
    expected = answer[4][0]
    predicted = answer[5][0]
    
    # index by spec_id and prop_code, since not all specs have all properties
    major_key = '%s-%s-%s-%s' % (spec_id, prop_code, spec_polarity, expected)
    if not major_key in freq_majority_votes:
        freq_majority_votes[major_key] = {'T': 0, 'F': 0, 'N': 0}
    
    # if expected == predicted
    if expected == predicted:
        freq_overall['T'] += 1
        freq_property[prop_code]['T'] += 1 
        freq_specification_polarity[spec_polarity]['T'] += 1
        freq_requirement_polarity[expected]['T'] += 1
        
    else: # expected != predicted
        freq_overall['F'] += 1
        freq_property[prop_code]['F'] += 1 
        freq_specification_polarity[spec_polarity]['F'] += 1
        freq_requirement_polarity[expected]['F'] += 1
    
    freq_majority_votes[major_key][predicted] += 1
        
print('Analyzed %i scenarios.' % len(scenario_ids))
print('Analyzed %i properties.' % len(properties))

Analyzed 304 scenarios.
Analyzed 8 properties.


## Compute Tie-Break Probability

The Tie-Break probability is the likelihood of choosing True or False in a tie break wherein an equal number of votes were tallied for each response option. This should be nearly equal to 50%, since this is equivalent to a random coin flip. Large deviations from 50% could be a source of measurement bias.

In [5]:
prob_dist = {'T': 0, 'F': 0}
sum_dist = 0

for major_key, votes in freq_majority_votes.items(): 
    majority = 'F'
    if votes['T'] > votes['F']:
        majority = 'T'
    elif votes['T'] == votes['F']:
        majority = random.sample(['T', 'F'], 1)[0]
        prob_dist[majority] += 1
        sum_dist += 1
            
    #print('%s %s: %i T <> %i F' % (major_key, major_key[-1], votes['T'], votes['F']))

    if majority == major_key[-1]:
        freq_majority['T'] += 1
    else:
        freq_majority['F'] += 1
        
if sum_dist > 0:        
    print('Tie-Break T Probability: %0.3f (~50%%)' % (prob_dist['T'] / (prob_dist['T'] + prob_dist['F'])))
else:
    print('No ties in voting detected.')


Tie-Break T Probability: 0.333 (~50%)


In [6]:
print('Overall Accuracy: %0.3f' % (freq_overall['T'] / (freq_overall['T'] + freq_overall['F'])))

Overall Accuracy: 0.761


## RQ1: To what extent does property affect accuracy?

In [7]:

print_order = ['P', 'C', 'G', 'D', 'S', 'I', 'U', 'W']
for prop_code in print_order:
    counts = freq_property[prop_code]
    print('Accuracy %s: %0.3f' % ( 
        prop_code, 
        counts['T'] / (counts['T'] + counts['F'])
    ))

Accuracy P: 0.876
Accuracy C: 0.761
Accuracy G: 0.676
Accuracy D: 0.849
Accuracy S: 0.730
Accuracy I: 0.672
Accuracy U: 0.675
Accuracy W: 0.846


## RQ2: To what extent does only specification polarity affect accuracy?

In [8]:
print('Accuracy T: %0.3f' % (
    freq_specification_polarity['T']['T'] / (freq_specification_polarity['T']['T'] + freq_specification_polarity['T']['F'])
))
print('Accuracy F: %0.3f' % (
    freq_specification_polarity['F']['T'] / (freq_specification_polarity['F']['T'] + freq_specification_polarity['F']['F'])
))

Accuracy T: 0.889
Accuracy F: 0.486


## RQ3: To what extent does only requirement polarity affect accuracy?

In [9]:
print('Accuracy T: %0.3f' % (
    freq_requirement_polarity['T']['T'] / (freq_requirement_polarity['T']['T'] + freq_requirement_polarity['T']['F'])
))
print('Accuracy F: %0.3f' % (
    freq_requirement_polarity['F']['T'] / (freq_requirement_polarity['F']['T'] + freq_requirement_polarity['F']['F'])
))

Accuracy T: 0.633
Accuracy F: 0.889


## RQ4: To what extent does majority vote affect accuracy?

In [10]:
print('Accuracy Majority Voting: %0.3f' % (
    freq_majority['T'] / (freq_majority['T'] + freq_majority['F'])
))

Accuracy Majority Voting: 0.783


In [11]:

print_order = ['P', 'C', 'G', 'D', 'S', 'I', 'U', 'W']
for prop_code in print_order:
    counts = freq_property[prop_code]
    print('%s\t%0.3f' % ( 
        prop_code, 
        counts['T'] / (counts['T'] + counts['F'])
    ))
    
print()
print('Spec (true) \t\t%0.3f' % (
    freq_specification_polarity['T']['T'] / (freq_specification_polarity['T']['T'] + freq_specification_polarity['T']['F'])
))
print('Spec (false)\t\t%0.3f' % (
    freq_specification_polarity['F']['T'] / (freq_specification_polarity['F']['T'] + freq_specification_polarity['F']['F'])
))

print('Req.\t\t\t%0.3f' % (
    freq_requirement_polarity['T']['T'] / (freq_requirement_polarity['T']['T'] + freq_requirement_polarity['T']['F'])
))
print('Req. inverted\t\t%0.3f' % (
    freq_requirement_polarity['F']['T'] / (freq_requirement_polarity['F']['T'] + freq_requirement_polarity['F']['F'])
))
    
print('Overall Acc.\t\t%0.3f' % (freq_overall['T'] / (freq_overall['T'] + freq_overall['F'])))

print('Self-consistency\t%0.3f' % (
    freq_majority['T'] / (freq_majority['T'] + freq_majority['F'])
))

P	0.876
C	0.761
G	0.676
D	0.849
S	0.730
I	0.672
U	0.675
W	0.846

Spec (true) 		0.889
Spec (false)		0.486
Req.			0.633
Req. inverted		0.889
Overall Acc.		0.761
Self-consistency	0.783
